<h1>Giao diện dự báo thời tiết</h1>

In [4]:
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np
import pickle

# ------------------------------
# 1. Load mô hình và LabelEncoder đã huấn luyện từ file
# ------------------------------
with open("weather_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# Định nghĩa thứ tự các đặc trưng (để đảm bảo nhất quán khi dự đoán)
feature_columns = ["Temp", "Feels", "Gust", "Rain", "Humidity", "Cloud", "Pressure", "Wind Speed"]

# Từ điển ánh xạ tên thời tiết từ tiếng Anh sang tiếng Việt
weather_vn = {
    "Clear": "Trời quang đãng",
    "Cloudy": "Trời nhiều mây",
    "Mist": "Sương mù",
    "Overcast": "Trời u ám",
    "Rain": "Mưa",
    "Sunny": "Nắng"
}

# ------------------------------
# 2. Xây dựng giao diện (GUI) dự báo
# ------------------------------
def center_window(win):
    """Căn giữa cửa sổ win trên màn hình."""
    win.update_idletasks()
    width = win.winfo_width()
    height = win.winfo_height()
    x = (win.winfo_screenwidth() - width) // 2
    y = (win.winfo_screenheight() - height) // 2
    win.geometry(f"{width}x{height}+{x}+{y}")

def show_custom_info(title, message):
    """Hiển thị kết quả dự đoán trong cửa sổ mới với chữ to và căn giữa màn hình.
       Khi bấm OK, cửa sổ kết quả đóng và các ô nhập liệu bị xóa."""
    top = tk.Toplevel(root)
    top.title(title)
    top.configure(bg='white')
    tk.Label(top, text=message, fg='black', bg='white', font=("Helvetica", 16)).pack(pady=20, padx=20)
    # Nút OK gọi hàm clear_inputs để xóa các ô nhập liệu, sau đó đóng cửa sổ kết quả
    tk.Button(top, text="OK", command=lambda: [clear_inputs(), top.destroy()],
              font=("Helvetica", 16, "bold")).pack(pady=10)
    center_window(top)

def predict_weather():
    """Dự đoán thời tiết từ dữ liệu người dùng nhập vào."""
    try:
        features = [
            float(entry_temp.get()),
            float(entry_feels.get()),
            float(entry_gust.get()),
            float(entry_rain.get()),
            float(entry_humidity.get()),
            float(entry_cloud.get()),
            float(entry_pressure.get()),
            float(entry_wind_speed.get())
        ]
        features = np.array([features])
        prediction = model.predict(features)[0]
        predicted_label = label_encoder.inverse_transform([prediction])[0]
        # Chuyển đổi kết quả dự đoán sang tiếng Việt
        predicted_label_vn = weather_vn.get(predicted_label, predicted_label)
        show_custom_info("Kết quả", f"Dự đoán: {predicted_label_vn}")
    except ValueError:
        messagebox.showerror("Lỗi", "Vui lòng nhập giá trị hợp lệ.")

# Hàm xóa các ô nhập liệu
def clear_inputs():
    for entry in entries.values():
        entry.delete(0, tk.END)

# Tạo cửa sổ chính
root = tk.Tk()
root.title("Dự đoán Thời Tiết")
root.geometry("800x400")
root.configure(bg="#e6f7ff")

# Cấu hình style cho các widget (chữ to)
style = ttk.Style()
style.configure("TLabel", background="#e6f7ff", font=("Helvetica", 16))
style.configure("TButton", background="#0078D7", foreground="black",
                font=("Helvetica", 16, "bold"), padding=5)
style.map("TButton", background=[("active", "#005a9e")],
          foreground=[("active", "black")])

# Tạo frame chính để căn giữa nội dung
main_frame = tk.Frame(root, bg="#e6f7ff")
main_frame.place(relx=0.5, rely=0.5, anchor="center")

# Danh sách các trường nhập liệu (mỗi dòng gồm nhãn và ô nhập kèm đơn vị)
fields = [
    ("Nhiệt độ (Temp) (°C):",         "entry_temp"),
    ("Cảm giác (Feels) (°C):",        "entry_feels"),
    ("Gió giật (Gust) (m/s):",        "entry_gust"),
    ("Mưa (Rain) (mm):",              "entry_rain"),
    ("Độ ẩm (Humidity) (%):",         "entry_humidity"),
    ("Mây (Cloud) (%):",              "entry_cloud"),
    ("Áp suất (Pressure) (hPa):",     "entry_pressure"),
    ("Tốc độ gió (Wind Speed) (m/s):","entry_wind_speed")
]

entries = {}
for i, (label_text, var_name) in enumerate(fields):
    lbl = ttk.Label(main_frame, text=label_text)
    lbl.grid(row=i, column=0, padx=5, pady=5, sticky="e")
    ent = ttk.Entry(main_frame, width=15, font=("Helvetica", 16))
    ent.grid(row=i, column=1, padx=5, pady=5, sticky="w")
    entries[var_name] = ent

# Gán biến cho từng Entry để sử dụng trong hàm dự đoán
entry_temp = entries["entry_temp"]
entry_feels = entries["entry_feels"]
entry_gust = entries["entry_gust"]
entry_rain = entries["entry_rain"]
entry_humidity = entries["entry_humidity"]
entry_cloud = entries["entry_cloud"]
entry_pressure = entries["entry_pressure"]
entry_wind_speed = entries["entry_wind_speed"]

# Nút "Dự đoán" căn giữa ở dưới cùng
btn_predict = ttk.Button(main_frame, text="Dự đoán", command=predict_weather)
btn_predict.grid(row=len(fields), column=0, columnspan=2, pady=20)

root.mainloop()


C:\Users\trand\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
